In [1]:
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [3]:
result = session.execute("""USE shop""").one()
if result:
    print(result[0])

In [4]:
result = session.execute("""
CREATE TABLE IF NOT EXISTS orders (
  name text,
  created_at date,
  amount int,
  items list<int>,
  PRIMARY KEY (name, created_at)
)
WITH CLUSTERING ORDER BY (created_at ASC);
""")

In [5]:
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Vadym',
          '2023-03-01',
          1600,
          [1, 2, 3]
         );
""")
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Vadym',
          '2023-02-01',
          100,
          [4]
         );
""")
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Bohdan',
          '2023-02-05',
          800,
          [1, 5]
         );
""")
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Bohdan',
          '2023-03-13',
          1390,
          [3, 6]
         );
""")
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Andriy',
          '2023-03-24',
          2100,
          [6, 7]
         );
""")

1) Напишіть запит, який показує структуру створеної таблиці (команда DESCRIBE)

In [6]:
result = session.execute("""DESCRIBE TABLE orders;
""")
if result:
    for row in result:
        print(*row, '\n')

shop table orders CREATE TABLE shop.orders (
    name text,
    created_at date,
    amount int,
    items list<int>,
    PRIMARY KEY (name, created_at)
) WITH CLUSTERING ORDER BY (created_at ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p'; 



2) Для замовника виведіть всі його замовлення відсортовані за часом коли вони були зроблені

In [7]:
result = session.execute("""
SELECT *
FROM orders
WHERE name = 'Vadym'
ORDER BY created_at;
""")
if result:
    for row in result:
        print(*row, '\n')

Vadym 2023-02-01 100 [4] 

Vadym 2023-03-01 1600 [1, 2, 3] 



3) Для замовника знайдіть замовлення з певним товаром

In [8]:
session.execute("""CREATE INDEX items_idx ON shop.orders (items);
""")

In [9]:
result = session.execute("""
SELECT *
FROM orders
WHERE name = 'Vadym'
  and items contains 2;
""")
if result:
    for row in result:
        print(*row, '\n')

Vadym 2023-03-01 1600 [1, 2, 3] 



4) Для замовника знайдіть замовлення за певний період часу і їх кількість

In [10]:
result = session.execute("""
SELECT *
FROm orders
WHERE name = 'Vadym'
  and created_at > '2023-01-10'
  and created_at < '2023-03-01';
""")
if result:
    for row in result:
        print(*row, '\n')

Vadym 2023-02-01 100 [4] 



In [11]:
result = session.execute("""
SELECT count(*)
FROm orders
WHERE name = 'Vadym'
  and created_at > '2023-01-10'
  and created_at < '2023-03-01';
""")
if result:
    for row in result:
        print(*row, '\n')

1 



5) Для кожного замовників визначте суму на яку були зроблені усі його замовлення

In [12]:
result = session.execute("""
SELECT name, sum(amount) as total_amount
FROM orders
GROUP BY name;
""")
if result:
    for row in result:
        print(*row, '\n')

Andriy 2100 

Bohdan 2190 

Vadym 1700 



6) Для кожного замовників визначте замовлення з максимальною вартістю

In [13]:
result = session.execute("""
SELECT name, max(amount)  as max_amount, created_at, items
FROM orders
GROUP BY name;
""")
if result:
    for row in result:
        print(*row, '\n')

Andriy 2100 2023-03-24 [6, 7] 

Bohdan 1390 2023-02-05 [1, 5] 

Vadym 1600 2023-02-01 [4] 



7) Модифікуйте певне замовлення додавши / видаливши один або кілька товарів при цьому також змінюючи вартість замовлення

In [14]:
session.execute("""
UPDATE orders
SET items = items + [8],
    amount = 1870
WHERE name = 'Vadym'
  and created_at = '2023-02-01';
""")

In [15]:
result = session.execute("""
SELECT *
FROM orders
WHERE name = 'Vadym'
  and created_at = '2023-02-01';
""")
if result:
    for row in result:
        print(*row, '\n')

Vadym 2023-02-01 1870 [4, 8] 



8) Для кожного замовлення виведіть час коли його ціна були занесена в базу (SELECT WRITETIME)

In [16]:
result = session.execute("""
SELECT name, created_at, amount, items, WRITETIME (amount)
FROM orders;
""")
if result:
    for row in result:
        print(*row, '\n')

Andriy 2023-03-24 2100 [6, 7] 1682426433158742 

Bohdan 2023-02-05 800 [1, 5] 1682426433154961 

Bohdan 2023-03-13 1390 [3, 6] 1682426433156951 

Vadym 2023-02-01 1870 [4, 8] 1682426944509493 

Vadym 2023-03-01 1600 [1, 2, 3] 1682426433147173 



9) Створіть замовлення з певним часом життя (TTL), після якого воно видалиться

In [17]:
session.execute("""
INSERT INTO orders (
    name, created_at, amount, items
) VALUES (
          'Andriy',
          '2023-03-30',
          1265,
          [1, 10]
         )
USING TTL 60;
""")

In [18]:
result = session.execute("""
SELECT *
FROM orders
WHERE name = 'Andriy'
  and created_at = '2023-03-30';
""")
if result:
    for row in result:
        print(*row, '\n')

Andriy 2023-03-30 1265 [1, 10] 



10) Поверніть замовлення у форматі JSON

In [19]:
result = session.execute("""
SELECT json *
FROM orders;
""")
if result:
    for row in result:
        print(*row, '\n')

{"name": "Andriy", "created_at": "2023-03-24", "amount": 2100, "items": [6, 7]} 

{"name": "Andriy", "created_at": "2023-03-30", "amount": 1265, "items": [1, 10]} 

{"name": "Bohdan", "created_at": "2023-02-05", "amount": 800, "items": [1, 5]} 

{"name": "Bohdan", "created_at": "2023-03-13", "amount": 1390, "items": [3, 6]} 

{"name": "Vadym", "created_at": "2023-02-01", "amount": 1870, "items": [4, 8]} 

{"name": "Vadym", "created_at": "2023-03-01", "amount": 1600, "items": [1, 2, 3]} 



11) Додайте замовлення у форматі JSON

In [20]:
session.execute("""
INSERT INTO orders JSON '{
  "name": "Vadym",
  "created_at": "2023-03-06",
  "amount": 845,
  "items": [6, 12]
}';
""")

In [21]:
result = session.execute("""
SELECT *
FROM orders;
""")
if result:
    for row in result:
        print(*row, '\n')

Andriy 2023-03-24 2100 [6, 7] 

Andriy 2023-03-30 1265 [1, 10] 

Bohdan 2023-02-05 800 [1, 5] 

Bohdan 2023-03-13 1390 [3, 6] 

Vadym 2023-02-01 1870 [4, 8] 

Vadym 2023-03-01 1600 [1, 2, 3] 

Vadym 2023-03-06 845 [6, 12] 

